# Session 03: Introduction to AFW Display

<br>Owner(s): **Keith Bechtol** ([@bechtol](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@bechtol))
<br>Last Verified to Run: **2020-05-28**
<br>Verified Stack Release: **w_2020_21**

In [Session 02](https://github.com/LSSTScienceCollaborations/StackClubCourse/blob/master/Session02/IntroToDataProductsAndTasks.ipynb), we learned how to access pixel-level astronomical images using the Stack, including the mask plane, variance plane, background, and additional associated information such as the WCS solution. We also briefly introduced [lsst.afw.display](https://pipelines.lsst.io/modules/lsst.afw.display/index.html) library for image visualization.

In this notebook, we 

1. point out additional image display features of `lsst.awf.display`,
2. interactively explore images with [Firefly](http://web.ipac.caltech.edu/staff/shupe/display_firefly),
3. and show how to create a color postage stamp. 

This tutorial is largely based on the [tutorial](https://pipelines.lsst.io/getting-started/display.html) on `pipelines.lsst.io`, the SQuARE Firefly [notebook](https://github.com/lsst-sqre/notebook-demo/blob/master/Firefly.ipynb), as well as the [demo](https://github.com/RobertLuptonTheGood/notebooks/blob/master/Demos/Colour%20Images.ipynb) by Robert Lupton. It is also recommended to check out the [AFW_Display_Demo.ipynb](https://github.com/LSSTScienceCollaborations/StackClub/blob/master/Visualization/AFW_Display_Demo.ipynb) Stack Club notebook by Brant Robertson. We combine lessons from these sources in an attempt to provide a coherent view of the image display capabilities in the Stack.

In [ ]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list -s lsst_distrib

## Additional Features of AFW Display

Begin by accessing the same exposure and source catalog from Session 02.

In [ ]:
REPO = '/datasets/hsc/repo/rerun/RC/w_2020_19/DM-24822'  
import lsst.daf.persistence as dafPersist
butler = dafPersist.Butler(REPO)

In [ ]:
VISIT = 34464
CCD = 81
exposure = butler.get('calexp', visit=int(VISIT), ccd=CCD)
src = butler.get('src', visit=int(VISIT), ccd=CCD)

But wait - how did we know what data existed in the repo? We can use `queryMetadata` to find out what combination of visits and ccds exist as data ids in the repo.

In [ ]:
# Example query
#butler.queryMetadata('calexp', ['visit', 'ccd'], dataId={'filter': 'HSC-R'})

In [ ]:
# Enable interactive widgets
%matplotlib widget

In [ ]:
import lsst.afw.display as afwDisplay

We can choose between multiple possible backends for afw_display, including matplotlib, ds9, and Firefly. Let's start with matplotlib. 

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
display = afwDisplay.Display(frame=0)
display.scale("linear", "zscale")
#display.scale("asinh", "zscale")
#display.setMaskTransparency(60) 
display.mtv(exposure)

We can overlay a scatter plot on the image to indicate the positions of detected sources.

In [ ]:
with display.Buffering():
    for s in src:
        display.dot("o", s.getX(), s.getY(), size=10, ctype='orange')

Erase the markers.

In [ ]:
#display.erase()

If we zoom in on a bright star in the image above, we can see a colorful set of masks overlaid on the image. To interpet these colors, we can check the mask plane definitions.

In [ ]:
mask = exposure.getMask()
for mask_name, mask_bit in mask.getMaskPlaneDict().items():
    print('{:20}: {}'.format(mask_name, display.getMaskPlaneColor(mask_name)))

In order to create a multipanel figure in matplotlib, or put a display into a particular `Axes`, one can use the `sca` (set current Axes) command. The following cell provides an example, and also shows some of the optional arguments that can be passed to control the appearance of the display.

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 3, figsize=(9, 4))
display = []
plt.sca(ax[0])
display.append(afwDisplay.Display(frame=fig))
display[0].scale("linear", "zscale")
display[0].mtv(exposure)
plt.sca(ax[1])
display.append(afwDisplay.Display(frame=fig))
display[1].scale("asinh", "zscale")
display[1].mtv(exposure)
plt.sca(ax[2])
display.append(afwDisplay.Display(frame=fig))
display[2].scale("linear", min=0, max=10)
display[2].mtv(exposure)

## Interactive Visualization with Firefly

To emphasize the concept that afw_display can work with different backends, we now create an interactive visualization with the same data using Firefly.

In [ ]:
display = afwDisplay.Display(frame=1, backend='firefly')

In [ ]:
display.mtv(exposure)

# We could overlay points, but instead, let's show how to do linked brushing
#with display.Buffering():
#    for record in src:
#        display.dot('o', record.getX(), record.getY(), size=20, ctype='orange')

Upload a table to allow linking between image, table, and plots.

In [ ]:
fc = display.getClient()
import firefly_client.plot as ffplt
ffplt.use_client(fc)
tbl_id = ffplt.upload_table(src, title='Source Catalog')

Create an additional linked scatter plot.

In [ ]:
ffplt.scatter(x_col='base_CircularApertureFlux_12_0_instFlux/base_GaussianFlux_instFlux',
              y_col='log10(base_CircularApertureFlux_12_0_instFlux)',
              size=4,
              color='blue',
              title='test ap flux/model mag vs. log(ap flux)',
              xlabel='Model',
              ylabel='Ap/Model')

One can also access the url to view the Firefly display in a web browser. This step should be done before creating the display (i.e., one would run the cell below before the preceeding cells). 

In [ ]:
#display.getClient().get_firefly_url()

## Color Postage Stamps

Next we make a color postage stamp. Robert Lupton picked out a dramatic lensed arc for this example.

In [ ]:
import lsst.afw.display.rgb as afwRgb
import lsst.geom
import lsst.afw.image as afwImage

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 5)

In [ ]:
data_path = "/project/rhl/Data/hsc-v13_0"

butler = dafPersist.Butler(data_path)
skymap = butler.get("deepCoadd_skyMap")

Start from a known coordinate of interest.

In [ ]:
ra, dec, name = 215.9747, -0.4344, "Lens"

ra_dec = lsst.geom.SpherePoint(ra*lsst.geom.degrees, dec*lsst.geom.degrees)

filters = "grizy"  # filters to process -- we choose our bands when we set B, R, G = ...

cutout_size = 500   # pixels

Locate the tract and patch that contains that coordinate.

In [ ]:
for i, tp in enumerate(skymap.findTractPatchList([ra_dec])):
    tractInfo, patchInfo = tp
    tract = tractInfo.getId()
    patch = "%d,%d" % patchInfo[0].getIndex()
    print(i, tract, patch)

Obtain the postage stamp from the deep coadd image in each band.

In [ ]:
images = {}
cutout_size = lsst.geom.ExtentI(300, 300)

for f in filters:
    filter_name = "HSC-%s" % f.upper()
    wcs = butler.get("deepCoadd_calexp_wcs", immediate=True,
                    tract=tract, patch=patch, filter=filter_name)
    xy = lsst.geom.PointI(wcs.skyToPixel(ra_dec))

    bbox = lsst.geom.BoxI(xy - cutout_size//2, cutout_size)

    images[f] = butler.get("deepCoadd_calexp_sub", bbox=bbox, immediate=True,
                            tract=tract, patch=patch, filter=filter_name).getMaskedImage()

Three different combinations of bands used as input to the Red-Green-Blue color image.

In [ ]:
rgbFileFmt = "%s-%%s.png" % name if False else None
if not False:
    min = dict(gri=0.01, riz=0.01, izy=0.01)
    max = dict(gri=0.20, riz=0.20, izy=0.25)
else:
    min = dict(gri=0.01, riz=0.01, izy=0.05)
    max = dict(gri=0.20, riz=0.40, izy=0.50)

Q = 10

#plt.close('all')
for bands in ["gri", "riz", "izy"]:
    plt.figure()
    B, G, R = bands
    rgb = afwRgb.makeRGB(images[R], images[G], images[B],
                         min[bands], max[bands] - min[bands], Q,
                         #saturatedBorderWidth=1, saturatedPixelValue=10
                         )
    
    afwRgb.displayRGB(rgb)
    #plt.title(f"{ra_dec} {bands}")
    
    # Optionally, write a file
    #if rgbFileFmt:
    #    afwRgb.writeRGB(rgbFileFmt % bands, rgb)